<a href="https://colab.research.google.com/github/dinhngoc267/NSEEN/blob/master/Final_NSEEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install -U strsimpy

     |████████████████████████████████| 51kB 1.9MB/s 


In [3]:
pip install faiss-cpu --no-cache

     |████████████████████████████████| 7.2MB 3.2MB/s 


In [4]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=3aeff0402b4c25d3852287ad36e426b356d541b3fb6814fae11b1c0b0ce8a565
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [5]:
import tensorflow as tf
from strsimpy.jaro_winkler import JaroWinkler
import xml.etree.ElementTree as ET
import os
import numpy as np
from itertools import combinations
import random
from keras.preprocessing.sequence import pad_sequences
import copy
import re
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from __future__ import absolute_import
from __future__ import print_function
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Embedding, Bidirectional, LSTM, InputLayer, GRU
from keras.optimizers import RMSprop
from keras import backend as K
from keras.optimizers import Adam
import math
import csv
import faiss
import nltk

Using TensorFlow backend.


time: 2.74 s


In [15]:
def load_reference_dict(data_dir):
  reference_dict = {};
  tsv_file = open(data_dir); 
  read_tsv = csv.reader(tsv_file, delimiter="\n")

  for row in read_tsv:
    arr = row[0].split(': '); 
    values = [];
    variant_names = arr[1].split('\t');
    
    for name in variant_names:
      values.append(name)  
    reference_dict[arr[0]] = values

  reference_dict_copy = copy.deepcopy(reference_dict) 
  return reference_dict, reference_dict_copy

def char_to_int_dict(reference_dict): 
  text = "" 
  for key, values in reference_dict.items(): 
    for value in values: 
      text = text + value 
  vocab = sorted(set(text)) 
  return dict((c,i+1) for i,c in enumerate(vocab))

def get_maxlen_sequence(reference_dict): 
  maxlen=0 
  for key, values in reference_dict.items(): 
    for value in values: 
      if maxlen < len(value): 
        maxlen = len(value) 
  return maxlen

def split_character(word): 
  return [char for char in word]

#convert char sequence to int sequence in reference set and pad sequence
def embedding_dataset(data,maxlen, char_to_int): 
  idx = 0 
  for values in data: 
    value_1 = [char_to_int[c] for c in values[0]]; 
    value_2 = [char_to_int[c] for c in values[1]];
    data[idx] = pad_sequences([value_1, value_2],maxlen,padding= 'post') 
    idx += 1; 
  return data;

def create_random_list(list_size, population):
  sample = [] 
  count = 0 
  while count < list_size: 
    index = np.random.randint(0, len(population), size=1) 
    if population[index[0]] != 0: 
      sample.append(index[0]) 
      population[index[0]] = 0 
      count += 1

  return sample, population

#Prepare 3 sub dataset
#1.Semantic
def create_sematic_pairs(reference_dict):
  #Positive pairs
  sematic_pairs = []; 
  labels = [];

  for key,values in reference_dict.items(): 
    lst = []; n = len(values); 
    for value in values[1:]:
      sematic_pairs.append((values[0],value));
      labels.append((1,1))
  print("There are ",len(sematic_pairs)," positive pairs in sematic set")

  #Negative sematic pairs
  refer_list = [];
  
  for key, values in reference_dict.items():
    refer_list.append(values);

  count = 0;
  for i in range(0, len(refer_list)-1):
    value_1 = refer_list[i][0];
    if len(refer_list[i+1]) >=3:
      #value_2 = random.sample(list(refer_list[i+1]), 3);
      sematic_pairs.append((value_1, refer_list[i+1][0]));
      sematic_pairs.append((value_1, refer_list[i+1][1]));
      sematic_pairs.append((value_1, refer_list[i+1][2]));
      labels.append((0,0));
      labels.append((0,0));
      labels.append((0,0));
      count += 3;
    if len(refer_list[i+1]) == 2:
      #value_2 = random.sample(list(refer_list[i+1]), 2);
      sematic_pairs.append((value_1, refer_list[i+1][0]));
      sematic_pairs.append((value_1, refer_list[i+1][1]));
      labels.append((0,0));
      labels.append((0,0));
      count += 2;
    if len(refer_list[i+1]) == 1:
      #value_2 = random.sample(list(refer_list[i+1]), 1);
      sematic_pairs.append((value_1, refer_list[i+1][0]));
      labels.append((0,0));
      count += 1;
  print("There are ",count ," negative pairs in sematic set") 
  return sematic_pairs, labels


def create_syntactic_pairs(reference_dict):

#Create Syntatic Variations
  syntactic_pairs = []; 
  labels = []; 
  
  #Same name syntactic
  for key, values in reference_dict.items(): 
    for value in values:
      modified_str = copy.deepcopy(value);
      #1. Find all sequence contain characters which are not alphanumerical. If '-' character then replace with ' '. Else remove.     
      if '-' in value:        
        modified_str = modified_str.replace('-', ' ');
      #2. Convert to lower cases.      
      if value.islower() == False: #and value.lower() not in values:
        modified_str = modified_str.lower();
      #3. Remove 's / s in  tions -> tion 
      if "'s" in value:
        modified_str = modified_str.replace("'s","");
      if "'" in value:
        modified_str = modified_str.replace("'","");        
      
      if modified_str != value and modified_str not in values:
        syntactic_pairs.append((value, modified_str));
        #print(values[0],',',modified_str);
  
  # Different name syntactic
    
    # Từ viết tắt: 
      # low density lipoprotin -> ldl
      # high density lipoprotein -> hdl
      # injection site -> is
      # injection site reaction -> isr/ isrs
      # Haemoglobin -> hb
      # Diastolic blood presures ->dbp
      # magnetic resonance imagining -> mri
      # Whilte blood cell -> WBC
      # absolute neutrophil count -> anc
      # greater than -> '>'
      # less than -> < / below
      # per -> / 
      # < -> less than
      # < -> below

    # Cú pháp: 
      # tions -> tion
      # than && values -> bỏ values 
      # number + times -> number*
      # increased ở cuối -> increase/increase in đầu
      # decreased cuối -> reduce đầu/ low đầu
      # infection cuối -> infections đầu

  for key, values in reference_dict.items():

    for value in values:       
      # low density lipoprotein -> ldl
      if 'low density lipoprotein' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("low density lipoprotein","ldl");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # high density lipoprotein -> hdl
      if 'high density lipoprotein' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("high density lipoprotein","hdl");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # injection site -> is
      if 'injection site' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("injection site","is");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # injection site reaction -> isr/ isrs
      if 'injection site reaction' in value.lower(): 
        modified_str = value.lower();
        modified_str_1 = modified_str.replace("injection site reaction","isr");

        if modified_str_1 not in values:
          syntactic_pairs.append((value, modified_str_1));
          #print(values[0],',',modified_str_1);

        modified_str_2 = modified_str.replace("injection site reaction","isrs");

        if modified_str_2 not in values:
          syntactic_pairs.append((value, modified_str_2));
          #print(values[0],',',modified_str_2);
      # Haemoglobin -> hb
      if 'haemoglobin' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("haemoglobin","hb");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # Diastolic blood pressure ->dbp
      if 'diastolic blood pressure' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("diastolic blood pressure","dbp");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # magnetic resonance imaging -> mri
      if 'magnetic resonance imaging' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("magnetic resonance imaging","mri");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # White blood cell -> wbc
      if 'white blood cell' in value.lower(): 
        modified_str = value.lower();
        modified_str = modified_str.replace("white blood cell","wbc");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # absolute neutrophil count -> anc
      if 'absolute neutrophil count' in value.lower():
        modified_str = value.lower(); 
        modified_str = modified_str.replace("absolute neutrophil count","anc");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # greater than -> '>'
      if 'greater than' in value.lower():
        modified_str = value.lower(); 
        modified_str = modified_str.replace("greater than",">");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # less than -> < / below
      if 'less than' in value.lower(): 
        modified_str = value.lower(); 
        modified_str_1 = modified_str.replace("less than","<");

        if modified_str_1 not in values:
          syntactic_pairs.append((value, modified_str_1));
          #print(values[0],',',modified_str_1);

        modified_str_2 = modified_str.replace("less than","below");

        if modified_str_2 not in values:
          syntactic_pairs.append((value, modified_str_2));
          #print(values[0],',',modified_str_2);
      # per -> / 
      if ' per ' in value.lower(): 
        modified_str = value.lower(); 
        modified_str = modified_str.replace(" per ","/");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # < -> less than/below
      if '<' in value.lower(): 
        modified_str = value.lower(); 
        modified_str_1 = modified_str.replace("<","less than");

        if modified_str_1 not in values:
          syntactic_pairs.append((value, modified_str_1));
          #print(values[0],',',modified_str_1);

        modified_str_2 = modified_str.replace("<","below");

        if modified_str_2 not in values:
          syntactic_pairs.append((value, modified_str_2));
          #print(values[0],',',modified_str_2);

      # than && values -> bỏ values 
      if ' than ' in value.lower() and ' values ' in value.lower():
        modified_str = value.lower(); 
        modified_str = modified_str.replace(" values ","");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);

      # number + times -> number*
      if ' times ' in value.lower():
        modified_str = value.lower(); 
        modified_str = modified_str.replace(" times ","*");

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # increased ở cuối -> increase/increase in đầu
      if 'increase' in value.split(" ")[-1]:
        arr = value.lower().split(" ");
        modified_str = "increases"
        for term in arr[0:len(arr)-1]:
          modified_str = modified_str + " " + term;

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
        
        modified_str = "increase in"
        for term in arr[0:len(arr)-1]:
          modified_str = modified_str + " " + term;

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);

      # decreased cuối -> reduce đầu/ low đầu
      if 'decrease' in value.split(" ")[-1]  :
        arr = value.lower().split(" ");
        modified_str = "reduce"
        for term in arr[0:len(arr)-1]:
          modified_str = modified_str + " " + term;

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
        
        modified_str = "low"
        for term in arr[0:len(arr)-1]:
          modified_str = modified_str + " " + term;

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);
      # infection cuối -> infections đầu
      if 'infection' in value.split(" ")[-1]:
        arr = value.lower().split(" ");
        modified_str = "infections"
        for term in arr[0:len(arr)-1]:
          modified_str = modified_str + " " + term;

        if modified_str not in values:
          syntactic_pairs.append((value, modified_str));
          #print(values[0],',',modified_str);


  #Calculate the similarity between pair 
  jarowinkler = JaroWinkler()
  count = 0;
  for i,pair in enumerate(syntactic_pairs): 
    similarity = jarowinkler.similarity(pair[0], pair[1])
    if (pair[0].isupper() or pair[1].isupper()) and pair[0].lower() == pair[1].lower() and len(pair[0]) <= 10:
      similarity = 0.99;
      count += 1;
    labels.append((similarity,1));
  print(count);

  print("There are", len(syntactic_pairs), "positive pairs in the syntactic set") 
  return syntactic_pairs, labels

import tensorflow
def cosine_distance(vects): 
  x,y=vects
  
  #x = K.l2_normalize(x, axis=-1) 
  #y = K.l2_normalize(y, axis=-1) 
  
  #return K.mean(1 - K.sum((x * y), axis=-1), axis=-1)
  sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))

def cos_dist_output_shape(shapes): 
  shape1, shape2 = shapes 
  return (shape1[0], 1)

def contrastive_loss(y, d):
  #d: distance; y: labels
  margin = 1 
  return K.mean((1-y) * K.square(d) + y*K.square(K.maximum(margin - d, 0)))

def create_base_network(input_shape): 
  model = Sequential()

  model.add(GRU(64,recurrent_dropout=0.2,return_sequences=True, input_shape=input_shape));
  model.add(Bidirectional(LSTM(64, return_sequences=True))) 
  model.add(Dropout(0.4))
  model.add(Bidirectional(LSTM(64, return_sequences=True))) 
  model.add(Dropout(0.4))
  model.add(Bidirectional(LSTM(64, return_sequences=True))) 
  model.add(Dropout(0.4))
  model.add(Bidirectional(LSTM(64, return_sequences=False))) 
  model.add(Dropout(0.4))
  model.add(Dense(128, activation='relu'))
  return model

def shuffle_data(data, labels): 
  indices = np.arange(data.shape[0]) # Splits the data into a training set and a

                               # validation set, but first shuffles the data,
                               # because you're deadling with data in which samples
                               # are ordered (all negative first, then all positive)
  np.random.shuffle(indices) 
  data = data[indices]
  labels = labels[indices] 
  return data, labels


#Define Siamese network
def Siamese_network(input_shape, dataset, labels, weights_file, save_dir): 
  base_network = create_base_network(input_shape);
  input_a = Input(shape=input_shape);
  input_b = Input(shape=input_shape);

  processed_a = base_network(input_a);
  processed_b = base_network(input_b);

  distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b]);
  model = Model([input_a, input_b], distance);

  #adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 

  if weights_file =="":
    model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.00005))

  # define the checkpoint
    filepath="/content/drive/My Drive/Colab Notebooks/NSEEN-weights-model-20.07.20-train_siamese_net-{epoch:02d}-{loss:.6f}.hdf5";
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1 , mode='min');
    callbacks_list = [checkpoint, myCallback(save_dir)];
    print(model);
    history = model.fit([dataset[:,0], dataset[:,1]], labels[:,0],
                  batch_size=512,
                  epochs=1,
                  callbacks=callbacks_list)
  else: 
    model.load_weights(weights_file) ;
    model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.001)) 
  return model;
import keras
class myCallback(keras.callbacks.Callback):
  def __init__(self, save_dir):
    self.max_acc = 0;
    self.save_dir = save_dir;
  # Khi kết thúc mỗi epoch cần embed lại reference set và predict lại evaluate data (biểu diễn lại chuỗi dựa vào mô hình học)
  def on_epoch_end(self, epoch, logs=None):
    model_base = self.model.layers[2];
    reference_embeddings = model_base.predict(reference_vecs);
    predicted_dev = model_base.predict(dev_set);

    #index = faiss.IndexFlatL2(256)   # build the index for cosine distance
    
    index = faiss.IndexFlatIP(128)   # build the index

    prev = copy.deepcopy(predicted_dev);
    reference = copy.deepcopy(reference_embeddings);
    faiss.normalize_L2(prev)
    faiss.normalize_L2(reference)
    index.add(reference);
    D, I = index.search(prev,10);

    #index.add(reference_embeddings) 

    #D,I = index.search(predicted_dev[0:], 10) # Trả về index trong mảng 1 chiều sau khi đã flatten. 
                                                      # Cần biết vị trí của vector trong tập reference. -> Dựa vào reference_vecs_pos
    result = [];
    list_id = [];
    for i in range(0,len(predicted_dev)):
      tmp = [];
      lst = [];
      for j in range(0,10):
        idx = I[i][j]      # I là mảng 2 chiều. vì chỉ đi tìm neighbors của 1 vector nên lấy dòng đầu tiên và cột thứ j
        tmp.append(reference_list[idx]);
        lst.append(reference_list_idx[idx][1]);
      result.append(tmp);
      list_id.append(lst)
    
    n_hit_1 = 0; n_hit_3 = 0; n_hit_5 = 0; n_hit_10 = 0;

    for i in range(0,len(result)):
      flag = False;
      for j in range(0,1):
        if evaluate_dev[i,2] == list_id[i][j]:
          flag = True;
          n_hit_1 += 1;
          n_hit_3 += 1;
          n_hit_5 += 1;
          n_hit_10 += 1;
          break;
      if flag == False:
        for j in range(1,3):
          if evaluate_dev[i,2] == list_id[i][j]:
            flag = True;
            n_hit_3 += 1;
            n_hit_5 += 1;
            n_hit_10 += 1;
            break;

        if flag == False:
          for j in range(3,5):
            if evaluate_dev[i,2] == list_id[i][j]:
              flag = True;
              n_hit_5 += 1;
              n_hit_10 += 1;
              break;
          
          if flag == False:
            for j in range(5,10):
              if evaluate_dev[i,2] == list_id[i][j]:
                flag = True;
                n_hit_10 += 1;
                break;

    print(n_hit_1, n_hit_3, n_hit_5, n_hit_10);
    avg_acc = (float((n_hit_1/len(predicted_dev))) + float((n_hit_3/len(predicted_dev))) + float((n_hit_5/len(predicted_dev))) + float((n_hit_10/len(predicted_dev))))/4;
    print('Average Accuray:', avg_acc*100,'%');



def flatten_dict(referece_dict): 
  reference_list = []; 
  reference_list_idx = [] # Save the index of term. 
  idx = 0; 
  for key, values in reference_dict.items(): 
    for value in values:
      reference_list.append(value) 
      reference_list_idx.append((idx, key)); 
      idx += 1;

  return (reference_list, reference_list_idx);



#cấu trúc dữ liệu: training_set = str, label drug, normalization_id,
def create_evaluate_data(dir): 
  evaluate_data = [];
  trees = []; #id in these file is int
  
  for fname in sorted(os.listdir(dir)): 
    if(fname[-4:] == '.xml'): 
      trees.append(ET.parse(os.path.join(dir,fname)))
  
  for tree in trees: 
    root = tree.getroot() 
    drug_label = root.attrib.get('drug');
    for reaction in root.iter('Reaction'):
      attribs = reaction.attrib    
      str_name = attribs.get('str')
      for normalization in reaction.iter('Normalization'):
        meddra_pt_id = normalization.attrib.get('meddra_pt_id');
        nor_id = normalization.attrib.get('id');
        if meddra_pt_id is None:         
          evaluate_data.append((str_name, drug_label, '0', nor_id));
        else:
          evaluate_data.append((str_name, drug_label, meddra_pt_id , nor_id));
  return evaluate_data
  

time: 2.46 s


In [11]:
#Create reference dictionary of entities from the xml files.
#The value of each key is a sequence of characters. 
print("Creating reference dictionary...")
reference_dict, reference_dict_copy = load_reference_dict("/content/drive/My Drive/Colab Datasets/reference.tsv")
print("Size of reference: ", len(reference_dict))

#Creating 3 training set.
print("Creating training set including: ")
print("1. Creating sematic set...")
(sematic_pairs, sematic_labels) = create_sematic_pairs(reference_dict)
print("2. Creating syntactic set...")
(syntactic_pairs, syntactic_labels) = create_syntactic_pairs(reference_dict = reference_dict_copy)
#Now, we need to transform the sequence of characters into vector of integers so that we can feed into the model.
#Here, I use char to index method to embedd the sequence into vector. 
print("Concatenating sematic and syntactic sets...")
dataset = sematic_pairs + syntactic_pairs;
labels = sematic_labels + syntactic_labels;

print("Embedding the dataset into vec of integers...")
maxlen = get_maxlen_sequence(reference_dict)
print("Max len sequence: ", maxlen)
char_to_int = char_to_int_dict(reference_dict)

Creating reference dictionary...
Size of reference:  21622
Creating training set including: 
1. Creating sematic set...
There are  55803  positive pairs in sematic set
There are  42581  negative pairs in sematic set
2. Creating syntactic set...
317
There are 81827 positive pairs in the syntactic set
Concatenating sematic and syntactic sets...
Embedding the dataset into vec of integers...
Max len sequence:  100
time: 4.89 s


In [12]:
(reference_list, reference_list_idx) = flatten_dict(reference_dict)
reference_vecs = copy.deepcopy(reference_list)
idx = 0;
for value in reference_vecs: 
  reference_vecs[idx] = [char_to_int[c] for c in value];    
  idx +=1;
reference_vecs = pad_sequences(reference_vecs,maxlen,padding= 'post')
reference_vecs  = reference_vecs / 84
reference_vecs = np.reshape(reference_vecs, reference_vecs.shape + (1,));

evaluate_dev = create_evaluate_data('/content/drive/My Drive/Colab Datasets/TAC2017/dev_xml');
evaluate_dev = np.array(evaluate_dev, dtype='object');

dev_set = copy.deepcopy(evaluate_dev[:,0])
idx = 0;
for value in dev_set: 
  dev_set[idx] = [char_to_int[c] for c in value];    
  idx +=1;
dev_set = pad_sequences(dev_set, maxlen, padding='post')
dev_set = dev_set / 84
dev_set = np.reshape(dev_set, dev_set.shape + (1,))

time: 6.56 s


In [13]:
training_set = sematic_pairs + syntactic_pairs;
training_labels = sematic_labels + syntactic_labels;

training_set = embedding_dataset(training_set, maxlen,char_to_int)
training_set = np.array(training_set, dtype='object');
training_set = training_set / 84;
training_labels = np.array(training_labels, dtype='object');
print("Shuffle data...")
(training_set, training_labels) = shuffle_data(training_set, training_labels);

print("Reshape data into the shape of (samples, timestep, features)...")
training_set = np.reshape(training_set, training_set.shape + (1,))

Shuffle data...
Reshape data into the shape of (samples, timestep, features)...
time: 7.96 s


In [14]:
tmp_set = copy.deepcopy(training_set);
tmp_labels = copy.deepcopy(training_labels);

time: 21 s


In [16]:
model = Siamese_network(training_set.shape[2:], training_set, training_labels,"",None);

Epoch 1/1
180211/180211 [==============================] - 1568s 9ms/step - loss: 0.2284

Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/NSEEN-weights-model-20.07.20-train_siamese_net-01-0.228425.hdf5
995 1187 1204 1207
Average Accuray: 68.96396396396396 %
time: 32min 11s


In [ ]:
evaluate_dev = create_evaluate_data('/content/drive/My Drive/Colab Datasets/TAC2017/gold_xml');
evaluate_dev = np.array(evaluate_dev, dtype='object');

dev_set = copy.deepcopy(evaluate_dev[:,0])
idx = 0;
for value in dev_set: 
  dev_set[idx] = [char_to_int[c] for c in value];    
  idx +=1;
dev_set = pad_sequences(dev_set, maxlen, padding='post')
dev_set = dev_set / 84
dev_set = np.reshape(dev_set, dev_set.shape + (1,))

time: 258 ms


In [18]:
model_base = model.layers[2];
reference_embeddings = model_base.predict(reference_vecs);

time: 5min 38s


In [19]:
predicted_dev = model_base.predict(dev_set);

time: 7.57 s


In [21]:
index = faiss.IndexFlatIP(128)   # build the index

reference = copy.deepcopy(reference_embeddings);
faiss.normalize_L2(reference)
index.add(reference);
dev = copy.deepcopy(predicted_dev);
faiss.normalize_L2(dev);
D, I = index.search(dev,10);

result = [];
list_id = [];
for i in range(0,len(predicted_dev)):
  tmp = [];
  lst = [];
  for j in range(0,10):
    idx = I[i][j]      # I là mảng 2 chiều. vì chỉ đi tìm neighbors của 1 vector nên lấy dòng đầu tiên và cột thứ j
    tmp.append(reference_list[idx]);
    lst.append(reference_list_idx[idx][1]);
  result.append(tmp);
  list_id.append(lst)

time: 1.64 s


In [25]:
for i, row in enumerate(evaluate_dev):
  for j in range(0,10):
    print(row[0],'-',result[i][j], D[i][j])

Streaming output truncated to the last 5000 lines.
papilloma - Pyorrhoea 0.9999996
papilloma - Symmastia 0.9999996
papilloma - Polypnoea 0.9999991
papilloma - migraines 0.9999991
papilloma - infection 0.99999905
papilloma - hematomas 0.99999905
papilloma - hematuria 0.99999905
papilloma - phlebitis 0.9999989
alopecia - alopecia 0.99999994
alopecia - Toxaemia 0.99999976
alopecia - Myxedema 0.9999996
alopecia - sciatica 0.99999946
alopecia - Trisomy 9 0.99999946
alopecia - Trisomy 8 0.9999994
alopecia - diarrhea 0.9999992
alopecia - Protein S 0.999999
alopecia - Progeria 0.9999988
alopecia - Trisomy 11 0.9999987
palmar-plantar erythrodysesthesia syndrome - palmar-plantar erythrodysesthesia syndrome 0.99999994
palmar-plantar erythrodysesthesia syndrome - Urinary control neurostimulator implantation 0.99999726
palmar-plantar erythrodysesthesia syndrome - Vomiting following gastrointestinal surgery 0.9999959
palmar-plantar erythrodysesthesia syndrome - Unspecified polyarthropathy or polyart

In [22]:
result

[['hypersensitivity reactions',
  'supraventricular arrhythmia',
  'neuropsychiatric symptoms',
  'photosensitivity reaction',
  'hypersensitivity reaction',
  'gastrointestinal symptoms',
  'supraventricular tachycardia',
  'Uvulopalatopharyngoplasty',
  'Retrospondylolisthesis NOS',
  'gastrointestinal toxicity'],
 ['dizziness',
  'mortality',
  'contusion',
  'posthitis',
  'myoclonus',
  'psychosis',
  'Synovitis',
  'nystagmus',
  'Dysyntery',
  'steatosis'],
 ['rash',
  'ates',
  'nmsc',
  'crnm',
  'ards',
  'Stye',
  'Kuru',
  'Yaws',
  'ptld',
  'pain'],
 ['pruritis',
  'myositis',
  'vulvitis',
  'cystitis',
  'tinnitus',
  'seizures',
  'myopathy',
  'asystole',
  'polyuria',
  'pruritus'],
 ['flushing',
  'wheezing',
  'drooling',
  'sneezing',
  'Tryptase',
  'insomnia',
  'erythema',
  'Oxytocin',
  'impetigo',
  'weakness'],
 ['headache',
  'Vitamin A',
  'Vitamin C',
  'Vitamin D',
  'cns ptld',
  'cns ptld',
  'Vaccinia',
  'Melalgia',
  'Vitamin E',
  'Normal EKG'],
 

time: 119 ms


In [ ]:
count = 0;
for i,row in enumerate(evaluate_dev):
  for j in range(0,1):
    if row[2] == list_id[i][j]:
      count += 1;
      break;
print(count)

4377
time: 9.56 ms


In [ ]:
count = 0;
hard_neg_pairs = [];
hard_neg_labels = [];
point = [];
for i in range(0, len(reference_embeddings)):
  count += 1; 
  closest_idx = I[i][1];
  cur_point = D[i][1];
  # Check if meddra_id is match:
  closest_meddra_id = reference_list_idx[closest_idx][1];
  current_meddra_id = reference_list_idx[i][1];
  if len(reference_dict[current_meddra_id]) == 1:
    continue;
  
  if current_meddra_id != closest_meddra_id:
    jarowinkler = JaroWinkler()
    similarity = jarowinkler.similarity(reference_list[i],reference_list[closest_idx])
    if similarity < 0.8:
      hard_neg_pairs.append((reference_list[i], reference_list[closest_idx]));
      point.append(cur_point);
      hard_neg_labels.append((0,0))
print('Found ', len(hard_neg_pairs),' hard negative pairs');

Found  57362  hard negative pairs
time: 3.61 s


In [ ]:
jarowinkler = JaroWinkler()
similarity = jarowinkler.similarity('Abdominal adhesions','Intestinal adhesions')
print(similarity)

0.7884711779448622
time: 2.78 ms


In [ ]:
count = 0;
hard_neg_pairs = [];
hard_neg_labels = [];
point = [];
for i in range(0, len(reference_embeddings)):
  count += 1; 
  closest_idx = I[i][1];
  cur_point = D[i][1];
  # Check if meddra_id is match:
  closest_meddra_id = reference_list_idx[closest_idx][1];
  current_meddra_id = reference_list_idx[i][1];
  if current_meddra_id != closest_meddra_id:
    hard_neg_pairs.append((reference_list[i], reference_list[closest_idx]));
    point.append(cur_point);
    hard_neg_labels.append((0,0))
print('Found ', len(hard_neg_pairs),' hard negative pairs');

Found  66333  hard negative pairs
time: 149 ms


In [ ]:
for i, pair in enumerate(hard_neg_pairs):
  print(pair, '- ', point[i]);
    

Streaming output truncated to the last 5000 lines.
('Respiratory syncytial virus serology', 'Lymphocyte stimulation test positive') -  0.9999916
('Respiratory syncytial virus test positive', 'Lymphoplasmacytoid lymphoma/immunocytoma') -  0.999997
('Respiratory syncytial virus serology positive', 'Villonodular synovitis involving ankle and foot') -  0.99999624
('RSV serology positive', 'Edema retroperitoneal') -  0.99993795
('Respiratory syncytial virus test negative', 'Myelodysplastic syndrome transformation') -  0.9999988
('Respiratory syncytial virus serology negative', 'Villonodular synovitis involving ankle and foot') -  0.99999774
('RSV serology negative', 'QT interval shortened') -  0.99997556
('Intestinal anastomosis complication', 'Prolactin-producing pituitary tumor') -  0.99999857
('Intestinal anastomosis stenosis', 'Pituitary gonadotrophin abnormal') -  0.99999434
('Acquired haemoglobinopathy', 'Secondary haemochromatosis') -  0.99999076
('Acquired hemoglobinopathy', 'Second

In [ ]:
count = 0;
hard_neg_pairs = [];
hard_neg_labels = [];
for i in range(0, len(reference_embeddings)):
  count += 1; 
  closest_idx = I[i][1];
  # Check if meddra_id is match:
  closest_meddra_id = reference_list_idx[closest_idx][1];
  current_meddra_id = reference_list_idx[i][1];
  if current_meddra_id != closest_meddra_id:
    tmp = [];
    for k in range(1,6):
      tmp_idx = I[i][k];
      tmp.append(reference_list[tmp_idx]);
    hard_neg_pairs.append((reference_list[i], ));
    hard_neg_labels.append((0,0))
print('Found ', len(hard_neg_pairs),' hard negative pairs');

Found  66413  hard negative pairs
time: 1.29 s


[('11-beta-hydroxylase deficiency',
  ['ABO haemolytic disease of newborn',
   'ECG abnormalities non-specific',
   'ABO hemolytic disease of newborn',
   'CSF lactate dehydrogenase normal',
   'C1 esterase deficiency acquired']),
 ('17 ketosteroids urine',
  ['CSF granulocyte count',
   'CSF pressure decreased',
   'HDL/cholesterol ratio',
   'CSF pressure increased',
   'B-cell lymphoma stage II']),
 ('17 ketosteroids urine decreased',
  ['ECG counterclockwise rotation',
   '17 ketosteroids urine increased',
   '17 ketosteroids urine abnormal',
   'HIV associated polyneuropathy',
   'HIV post exposure prophylaxis']),
 ('17 ketosteroids urine low',
  ['21-hydroxylase deficiency',
   '17 ketosteroids urine high',
   'CD8 lymphocytes increased',
   'B-cell lymphoma recurrent',
   'CD4 lymphocytes increased']),
 ('17 ketosteroids urine increased',
  ['ECG counterclockwise rotation',
   '17 ketosteroids urine decreased',
   '17 ketosteroids urine abnormal NOS',
   'B. pertussis serology n

time: 84.1 ms


In [ ]:
# Hard neg mining

D,I = index.search(reference_embeddings,2)
count = 0;
hard_neg_pairs = [];
hard_neg_labels = [];
for i in range(0, len(reference_embeddings)):
  count += 1; 
  closest_idx = I[i][1];
  # Check if meddra_id is match:
  closest_meddra_id = reference_list_idx[closest_idx][1];
  current_meddra_id = reference_list_idx[i][1];
  if current_meddra_id != closest_meddra_id:
    hard_neg_pairs.append((reference_list[i], reference_list[closest_idx]));
    hard_neg_labels.append((0,0))
print('Found ', len(hard_neg_pairs),' hard negative pairs');

# Lan 1:

tmp_set = sematic_pairs + syntactic_pairs + hard_neg_pairs
tmp_labels = sematic_labels + syntactic_labels + hard_neg_labels

training_set = copy.deepcopy(tmp_set);
training_labels = copy.deepcopy(tmp_labels);
training_set = embedding_dataset(training_set, maxlen,char_to_int)
training_set = np.array(training_set, dtype='object');
training_labels = np.array(training_labels, dtype='object');

print("Shuffle data...")
(training_set, training_labels) = shuffle_data(training_set, training_labels);

print("Reshape data into the shape of (samples, timestep, features)...")
training_set = np.reshape(training_set, training_set.shape + (1,));


filepath="/content/drive/My Drive/Colab Notebooks/NSEEN-weights-model-14.07.20-hard_neg_mining_of_epoch_4_dim256-{epoch:02d}-{loss:.6f}.hdf5";
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1 , mode='min');
callbacks_list = [checkpoint];

model.fit([training_set[:,0], training_set[:,1]], training_labels[:,0],
          batch_size=256,
          epochs=5,
          callbacks=callbacks_list)

model_base = model.layers[2];

reference_embeddings = embed_reference(reference_vecs, model_base) # BiLSTM tower.

index = faiss.IndexFlatL2(256)   # build the index
index.add(reference_embeddings) 
result = [];
list_id = [];

predict = model_base.predict(training_data)

D,I = index.search(predict[0:], 10) # Trả về index trong mảng 1 chiều sau khi đã flatten. 
                                                  # Cần biết vị trí của vector trong tập reference. -> Dựa vào reference_vecs_pos
for i in range(0,len(predict)):
  tmp = [];
  lst = [];
  for j in range(0,10):
    idx = I[i][j]      # I là mảng 2 chiều. vì chỉ đi tìm neighbors của 1 vector nên lấy dòng đầu tiên và cột thứ j
    tmp.append(reference_list[idx]);
    lst.append(reference_list_idx[idx][1]);
  result.append(tmp);
  list_id.append(lst)

count = 0;
for i in range(0,len(result)):
  flag = False;
  for j in range(0,10):
    if evaluate_data[i,2] == list_id[i][j]:
      flag = True;
      break;
  if flag==False:
    count += 1;
print(count)

